<h2> 3.8 Featurizing text data or Given SQL Queries </h2>

In [1]:
#importing essential libararies
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
from scipy.sparse import hstack
from scipy import sparse
import pickle
from sklearn.preprocessing import StandardScaler

In [2]:
#importing the preprocessed data
data = pd.read_csv('feature_extracted_1.csv')


In [3]:
data.drop('Unnamed: 0',axis = 1,inplace = True)

In [4]:
data.columns

Index(['Query', 'Label', 'query_len', 'num_words_query', 'no_single_qts',
       'no_double_qts', 'no_punct', 'no_single_cmnt', 'no_mult_cmnt',
       'no_space', 'no_perc', 'no_log_opt', 'no_arith', 'no_null', 'no_hexa',
       'no_alpha', 'no_digit', 'len_of_chr_char_null', 'genuine_keywords',
       'log_query_len', 'box_query_len'],
      dtype='object')

In [5]:
data.head()

,Query,Label,query_len,num_words_query,no_single_qts,no_double_qts,no_punct,no_single_cmnt,no_mult_cmnt,no_space,...,no_log_opt,no_arith,no_null,no_hexa,no_alpha,no_digit,len_of_chr_char_null,genuine_keywords,log_query_len,box_query_len
0,""" or pg_sleep ( __time__ ) --",1,33,7,0,1,10,1,0,6,...,1,2,0,0,13,0,0,0,3.496508,4.571114
1,create user name identified by pass123 tempora...,1,90,12,0,0,1,0,0,11,...,0,0,0,0,75,3,0,0,4.499810,6.378650
2,and 1 = utl_inaddr.get_host_address ( ...,1,218,35,3,0,25,0,0,35,...,2,0,0,0,120,2,0,2,5.384495,8.209386
3,select * from users where id = '1' or @ @1 ...,1,90,20,3,0,13,1,0,20,...,1,3,0,0,42,5,0,2,4.499810,6.378650
4,"select * from users where id = 1 or 1#"" ( ...",1,85,18,0,1,10,1,0,18,...,1,3,0,0,42,4,0,2,4.442651,6.268357


In [6]:
#dropping the query_len,num_words_query because from the EDA we got to know that it is not very much helpful in predicting the output classes
data.drop(['query_len','num_words_query'],axis= 1,inplace = True)

In [7]:

y = data['Label']
x = data.drop('Label',axis = 1)

<h2> 3.8.1 Splitting the dataset to train and test </h2>

In [8]:
#will split the datset in 70 and 30 i.e 70 percent for training and 30 percent for test
#since the class is imbalanced will do stratify sampling

#will remove the target variable i.e label column from datset

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,stratify = y)


print("train dataset size is {}".format(x_train.shape))
print("test dataset size is {}".format(x_test.shape))

train dataset size is (21523, 18)
test dataset size is (9225, 18)


In [9]:
#storing for future use
with open('x_train','wb') as f:
    pickle.dump(x_train,f)
with open('x_test','wb') as f:
    pickle.dump(x_test,f)
with open('y_train','wb') as f:
    pickle.dump(y_train,f)
with open('y_test','wb') as f:
    pickle.dump(y_test,f)


<h2> 3.8.2 Featurizing text data or Given SQL Queries using Bag of words </h2>

In [10]:
#using countvectorizer for bag of words
#using bag of words for unigram range

unigram_bow = CountVectorizer(ngram_range = (1,1))
train_bow = unigram_bow.fit(x_train['Query'].values)


print("After Encoding text data")
x_train_bow_unigram = train_bow.transform(x_train['Query'].values)
x_test_bow_unigram = train_bow.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_bow_unigram.shape))
print("the shape of test dataset unigram is  {}".format(x_test_bow_unigram.shape))



After Encoding text data
the shape of train dataset unigram is (21523, 18629)
the shape of test dataset unigram is  (9225, 18629)


In [11]:
with open('unigram_bow','wb') as f:
    pickle.dump(unigram_bow,f)

In [12]:
#saving train_bow for future use
with open('train_bow','wb') as f:
    pickle.dump(train_bow,f)

In [13]:
#will use bigram range of values to build bow
bigram_bow = CountVectorizer(ngram_range = (2,2))
train_bigram_bow = bigram_bow.fit(x_train['Query'].values)

print("After Encoding text data")
x_train_bow_bigram = train_bigram_bow.transform(x_train['Query'].values)
x_test_bow_bigram = train_bigram_bow.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_bow_bigram.shape))
print("the shape of test dataset unigram is  {}".format(x_test_bow_bigram.shape))


After Encoding text data
the shape of train dataset unigram is (21523, 51979)
the shape of test dataset unigram is  (9225, 51979)


<h2> 3.8.3 Featurizing text data or Given SQL Queries using tfidf vectorizer </h2>

In [14]:
#considering only unigrams
tfidf_bow = TfidfVectorizer(ngram_range = (1,1))
tfidf_train_bow = tfidf_bow.fit(x_train['Query'].values)


print("After Encoding text data")
x_train_tfidf_unigram = tfidf_train_bow.transform(x_train['Query'].values)
x_test_tfidf_unigram = tfidf_train_bow.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_tfidf_unigram.shape))
print("the shape of test dataset unigram is  {}".format(x_test_tfidf_unigram.shape))



After Encoding text data
the shape of train dataset unigram is (21523, 18629)
the shape of test dataset unigram is  (9225, 18629)


In [15]:
#considering only bigrams
tfidf = TfidfVectorizer(ngram_range = (2,2))
tfidf_train_bigram = tfidf.fit(x_train['Query'].values)


print("After Encoding text data")
x_train_tfidf_bigram = tfidf_train_bigram.transform(x_train['Query'].values)
x_test_tfidf_bigram = tfidf_train_bigram.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_tfidf_bigram.shape))
print("the shape of test dataset unigram is  {}".format(x_test_tfidf_bigram.shape))



After Encoding text data
the shape of train dataset unigram is (21523, 51979)
the shape of test dataset unigram is  (9225, 51979)


<h2> 3.8.4 Featurizing text data or Given SQL Queries using Average word2vec </h2>

<h3> 3.8.4.1 Using pretrained Glove vectors for words Embeddings </h3>

In [16]:
#loading glove vector file
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())
    

In [17]:
#compute average word2vec for each train data query
avg_w2v_vectors_train = [] 
for sentence in tqdm(x_train['Query'].values): 
    vector = np.zeros(300) 
    cnt_words =0
    for word in sentence.split(): 
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors_train.append(vector)

print(len(avg_w2v_vectors_train))
print(len(avg_w2v_vectors_train[0]))

100%|█████████████████████████████████████████████████████████████████████████| 21523/21523 [00:00<00:00, 56886.23it/s]

21523
300


In [18]:
#compute average word2vec for each test data query
avg_w2v_vectors_test = [] 
for sentence in tqdm(x_test['Query'].values): 
    vector = np.zeros(300) 
    cnt_words =0
    for word in sentence.split(): 
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors_test.append(vector)

print(len(avg_w2v_vectors_test))
print(len(avg_w2v_vectors_test[0]))

100%|███████████████████████████████████████████████████████████████████████████| 9225/9225 [00:00<00:00, 58992.02it/s]

9225
300


<h2> 3.8.5 Featurizing text data or Given SQL Queries using Tfidf weighted word2vec </h2>

In [19]:
tfidf_model = TfidfVectorizer()
tfidf_model.fit(x_train['Query'].values)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words = set(tfidf_model.get_feature_names())

In [20]:
#computing tfidf word2vec for each train data query
tfidf_w2v_vectors_train = []
for sentence in tqdm(x_train['Query'].values):
    vector = np.zeros(300)
    tf_idf_weight =0
    for word in sentence.split(): 
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] 
           
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) 
            vector += (vec * tf_idf)
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_train.append(vector)

print(len(tfidf_w2v_vectors_train))
print(len(tfidf_w2v_vectors_train[0]))

100%|█████████████████████████████████████████████████████████████████████████| 21523/21523 [00:00<00:00, 24452.04it/s]

21523
300


In [21]:
#computing tfidf word2vec for each test data query
tfidf_w2v_vectors_test = []
for sentence in tqdm(x_test['Query'].values):
    vector = np.zeros(300)
    tf_idf_weight =0
    for word in sentence.split(): 
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] 
           
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) 
            vector += (vec * tf_idf)
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_test.append(vector)

print(len(tfidf_w2v_vectors_test))
print(len(tfidf_w2v_vectors_test[0]))

100%|███████████████████████████████████████████████████████████████████████████| 9225/9225 [00:00<00:00, 25488.11it/s]

9225
300


<h2> 3.9 Encoding Numerical features </h2>

<h2> 3.9.1 Encoding Numerical feature : number of single line quotation marks </h2>

In [22]:
'''
Since we have numerical features will transform the numerical features using standardscaler method

'''
#Generalized function for standardscaler method
def transform(x_train,x_test):
    sc = StandardScaler()
    sc.fit(x_train.reshape(-1,1))
    train_d = sc.transform(x_train.reshape(-1,1))
    test_d = sc.transform(x_test.reshape(-1,1))
    return train_d,test_d


train_no_single_qts,test_no_single_qts  = transform(np.array(x_train['no_single_qts']).reshape(-1,1),np.array(x_test['no_single_qts']).reshape(-1,1))


print("the shape of no_single_qts after Encoding is {}".format(train_no_single_qts.shape))

the shape of no_single_qts after Encoding is (21523, 1)


<h2> 3.9.2 Encoding Numerical feature : number of double line quotation marks </h2>

In [23]:

train_no_double_qts,test_no_double_qts  = transform(np.array(x_train['no_double_qts']).reshape(-1,1),np.array(x_test['no_double_qts']).reshape(-1,1))

print("the shape of no_double_qts after Encoding is {}".format(train_no_double_qts.shape))

the shape of no_double_qts after Encoding is (21523, 1)


<h2> 3.9.3 Encoding Numerical feature : number of punctuations </h2>

In [24]:
#alpha value is choosen as 1 because if there are no elements present then denominator will be zero

train_no_punct,test_no_punct  = transform(np.array(x_train['no_punct']).reshape(-1,1),np.array(x_test['no_punct']).reshape(-1,1))

print("the shape of no_punct after Encoding is {}".format(test_no_punct.shape))

the shape of no_punct after Encoding is (9225, 1)


<h2> 3.9.4 Encoding Numerical feature : number of single line comments </h2>

In [25]:

train_no_single_cmnt,test_no_single_cmnt  = transform(np.array(x_train['no_single_cmnt']).reshape(-1,1),np.array(x_test['no_single_cmnt']).reshape(-1,1))

print("the shape of no_single_cmnt after Encoding is {}".format(train_no_single_cmnt.shape))

the shape of no_single_cmnt after Encoding is (21523, 1)


<h2> 3.9.5 Encoding Numerical feature : number of multi line comments </h2>

In [26]:

train_no_mult_cmnt,test_no_mult_cmnt = transform(np.array(x_train['no_mult_cmnt']).reshape(-1,1),np.array(x_test['no_mult_cmnt']).reshape(-1,1))

print("the shape of no_mult_cmnt after Encoding is {}".format(train_no_mult_cmnt.shape))

the shape of no_mult_cmnt after Encoding is (21523, 1)


<h2> 3.9.6 Encoding Numerical feature : number of space </h2>

In [27]:

train_no_space,test_no_space  = transform(np.array(x_train['no_space']).reshape(-1,1),np.array(x_test['no_space']).reshape(-1,1))

print("the shape of no_space after Encoding is {}".format(train_no_space.shape))

the shape of no_space after Encoding is (21523, 1)


<h2> 3.9.7 Encoding Numerical feature : number of percentage symbols </h2>

In [28]:

train_no_perc,test_no_perc  = transform(np.array(x_train['no_perc']).reshape(-1,1),np.array(x_test['no_perc']).reshape(-1,1))

print("the shape of no_perc after Encoding is {}".format(train_no_perc.shape))

the shape of no_perc after Encoding is (21523, 1)


<h2> 3.9.8 Encoding Numerical feature : number of logical operators </h2>

In [29]:

train_no_log_opt,test_no_log_opt  = transform(np.array(x_train['no_log_opt']).reshape(-1,1),np.array(x_test['no_log_opt']).reshape(-1,1))

print("the shape of no_log_opt after Encoding is {}".format(train_no_log_opt.shape))

the shape of no_log_opt after Encoding is (21523, 1)


<h2> 3.9.9 Encoding Numerical feature : number of arithmetic operators </h2>

In [30]:

train_no_arith_opt,test_no_arith_opt  = transform(np.array(x_train['no_arith']).reshape(-1,1),np.array(x_test['no_arith']).reshape(-1,1))

print("the shape of no_arith_opt after Encoding is {}".format(train_no_arith_opt.shape))

the shape of no_arith_opt after Encoding is (21523, 1)


<h2> 3.9.10 Encoding Numerical feature : number of null values </h2>

In [31]:

train_no_null,test_no_null  = transform(np.array(x_train['no_null']).reshape(-1,1),np.array(x_test['no_null']).reshape(-1,1))

print("the shape of no_null after Encoding is {}".format(train_no_null.shape))

the shape of no_null after Encoding is (21523, 1)


<h2> 3.9.10 Encoding Numerical feature : number of hexadecimal values </h2>

In [32]:

train_no_hexa,test_no_hexa = transform(np.array(x_train['no_hexa']).reshape(-1,1),np.array(x_test['no_hexa']).reshape(-1,1))

print("the shape of no_hexa after Encoding is {}".format(train_no_hexa.shape))

the shape of no_hexa after Encoding is (21523, 1)


<h2> 3.9.11 Encoding Numerical feature : number of alphabets </h2>

In [33]:

train_no_alpha,test_no_alpha = transform(np.array(x_train['no_alpha']).reshape(-1,1),np.array(x_test['no_alpha']).reshape(-1,1)) 

print("the shape of no_alpha after Encoding is {}".format(train_no_alpha.shape))

the shape of no_alpha after Encoding is (21523, 1)


<h2> 3.9.12 Encoding Numerical feature : number of digits </h2>

In [34]:

train_no_digit,test_no_digit = transform(np.array(x_train['no_digit']).reshape(-1,1),np.array(x_test['no_digit']).reshape(-1,1))

print("the shape of no_digit after Encoding is {}".format(train_no_digit.shape))

the shape of no_digit after Encoding is (21523, 1)


<h2> 3.9.13 Encoding Numerical feature : length of che,char and null keywords combined </h2>

In [35]:

train_len_of_chr_char_null,test_len_of_chr_char_null  = transform(np.array(x_train['len_of_chr_char_null']).reshape(-1,1),np.array(x_test['len_of_chr_char_null']).reshape(-1,1))

print("the shape of len_of_chr_char_null after Encoding is {}".format(train_len_of_chr_char_null.shape))

the shape of len_of_chr_char_null after Encoding is (21523, 1)


<h2> 3.9.14 Encoding Numerical feature : length of genuine_keywords </h2>

In [36]:

train_genuine_keywords,test_genuine_keywords  = transform(np.array(x_train['genuine_keywords']).reshape(-1,1),np.array(x_test['genuine_keywords']).reshape(-1,1))

print("the shape of genuine_keywords after Encoding is {}".format(train_genuine_keywords.shape))

the shape of genuine_keywords after Encoding is (21523, 1)


<h2> 4. Combining all the features encoded above</h2>

<h2> 4.1 Concatenating Encoded features with bag of words unigram range encoding</h2>

In [37]:
train_data_bow_unigram = hstack((x_train_bow_unigram,train_no_single_qts,train_no_double_qts,train_no_punct,train_no_single_cmnt,train_no_mult_cmnt,train_no_space,train_no_perc,train_no_log_opt,train_no_arith_opt,train_no_null,train_no_hexa,train_no_alpha,train_no_digit,train_len_of_chr_char_null,train_genuine_keywords)).tocsr()
test_data_bow_unigram = hstack((x_test_bow_unigram,test_no_single_qts,test_no_double_qts,test_no_punct,test_no_single_cmnt,test_no_mult_cmnt,test_no_space,test_no_perc,test_no_log_opt,test_no_arith_opt,test_no_null,test_no_hexa,test_no_alpha,test_no_digit,test_len_of_chr_char_null,test_genuine_keywords)).tocsr()

<h2> 4.2 Concatenating Encoded features with bag of words bigram range encoding</h2>

In [38]:
train_data_bow_bigram = hstack((x_train_bow_bigram,train_no_single_qts,train_no_double_qts,train_no_punct,train_no_single_cmnt,train_no_mult_cmnt,train_no_space,train_no_perc,train_no_log_opt,train_no_arith_opt,train_no_null,train_no_hexa,train_no_alpha,train_no_digit,train_len_of_chr_char_null,train_genuine_keywords)).tocsr()
test_data_bow_bigram = hstack((x_test_bow_bigram,test_no_single_qts,test_no_double_qts,test_no_punct,test_no_single_cmnt,test_no_mult_cmnt,test_no_space,test_no_perc,test_no_log_opt,test_no_arith_opt,test_no_null,test_no_hexa,test_no_alpha,test_no_digit,test_len_of_chr_char_null,test_genuine_keywords)).tocsr()

<h2> 4.3 Concatenating Encoded features with tfidf vectorizer unigram encoding</h2>

In [39]:
train_data_tfidf_unigram = hstack((x_train_tfidf_unigram,train_no_single_qts,train_no_double_qts,train_no_punct,train_no_single_cmnt,train_no_mult_cmnt,train_no_space,train_no_perc,train_no_log_opt,train_no_arith_opt,train_no_null,train_no_hexa,train_no_alpha,train_no_digit,train_len_of_chr_char_null,train_genuine_keywords)).tocsr()
test_data_tfidf_unigram = hstack((x_test_tfidf_unigram,test_no_single_qts,test_no_double_qts,test_no_punct,test_no_single_cmnt,test_no_mult_cmnt,test_no_space,test_no_perc,test_no_log_opt,test_no_arith_opt,test_no_null,test_no_hexa,test_no_alpha,test_no_digit,test_len_of_chr_char_null,test_genuine_keywords)).tocsr()

<h2> 4.4 Concatenating Encoded features with tfidf vectorizer bigram encoding</h2

In [40]:
train_data_tfidf_bigram = hstack((x_train_tfidf_bigram,train_no_single_qts,train_no_double_qts,train_no_punct,train_no_single_cmnt,train_no_mult_cmnt,train_no_space,train_no_perc,train_no_log_opt,train_no_arith_opt,train_no_null,train_no_hexa,train_no_alpha,train_no_digit,train_len_of_chr_char_null,train_genuine_keywords)).tocsr()
test_data_tfidf_bigram = hstack((x_test_tfidf_bigram,test_no_single_qts,test_no_double_qts,test_no_punct,test_no_single_cmnt,test_no_mult_cmnt,test_no_space,test_no_perc,test_no_log_opt,test_no_arith_opt,test_no_null,test_no_hexa,test_no_alpha,test_no_digit,test_len_of_chr_char_null,test_genuine_keywords)).tocsr()

<h2> 4.5 Concatenating Encoded features with Average word2vec</h2

In [41]:
train_data_avg_word2vec = np.hstack((np.array(avg_w2v_vectors_train),train_no_single_qts,train_no_double_qts,train_no_punct,train_no_single_cmnt,train_no_mult_cmnt,train_no_space,train_no_perc,train_no_log_opt,train_no_arith_opt,train_no_null,train_no_hexa,train_no_alpha,train_no_digit,train_len_of_chr_char_null,train_genuine_keywords))
test_data_avg_word2vec = np.hstack((np.array(avg_w2v_vectors_test),test_no_single_qts,test_no_double_qts,test_no_punct,test_no_single_cmnt,test_no_mult_cmnt,test_no_space,test_no_perc,test_no_log_opt,test_no_arith_opt,test_no_null,test_no_hexa,test_no_alpha,test_no_digit,test_len_of_chr_char_null,test_genuine_keywords))

<h2> 4.6 Concatenating Encoded features with tfidf word2vec</h2

In [42]:
train_data_tfidf_word2vec = np.hstack((np.array(tfidf_w2v_vectors_train),train_no_single_qts,train_no_double_qts,train_no_punct,train_no_single_cmnt,train_no_mult_cmnt,train_no_space,train_no_perc,train_no_log_opt,train_no_arith_opt,train_no_null,train_no_hexa,train_no_alpha,train_no_digit,train_len_of_chr_char_null,train_genuine_keywords))
test_data_tfidf_word2vec = np.hstack((np.array(tfidf_w2v_vectors_test),test_no_single_qts,test_no_double_qts,test_no_punct,test_no_single_cmnt,test_no_mult_cmnt,test_no_space,test_no_perc,test_no_log_opt,test_no_arith_opt,test_no_null,test_no_hexa,test_no_alpha,test_no_digit,test_len_of_chr_char_null,test_genuine_keywords))

In [43]:
#storing all features generated above for further use
sparse.save_npz("train_data_bow_unigram",train_data_bow_unigram)
sparse.save_npz("test_data_bow_bigram.npz",test_data_bow_bigram)
sparse.save_npz("train_data_tfidf_unigram.npz",train_data_tfidf_unigram)
sparse.save_npz("train_data_tfidf_bigram.npz",train_data_tfidf_bigram)


In [44]:
sparse.save_npz("test_data_bow_unigram",test_data_bow_unigram)
sparse.save_npz("train_data_bow_bigram.npz",train_data_bow_bigram)
sparse.save_npz("test_data_tfidf_unigram.npz",test_data_tfidf_unigram)
sparse.save_npz("test_data_tfidf_bigram.npz",test_data_tfidf_bigram)



In [45]:
x_train.to_csv('x_train.csv')
x_test.to_csv('x_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

In [46]:
#using pickle files to store word2vec numpy arrays
with open('train_data_avg_word2vec','wb') as f:
    pickle.dump(train_data_avg_word2vec,f)
    
with open('test_data_avg_word2vec','wb') as f:
    pickle.dump(test_data_avg_word2vec,f)
    
with open('train_data_tfidf_word2vec','wb') as f:
    pickle.dump(train_data_tfidf_word2vec,f)
    
with open('test_data_tfidf_word2vec','wb') as f:
    pickle.dump(test_data_tfidf_word2vec,f)